In [1]:
import numpy as np
import pandas as pd

from dataclasses import dataclass
from typing import Tuple, Dict

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import kagglehub
import yfinance as yf
from openai import OpenAI
import os


In [3]:
@dataclass
class PaperConfig:
    start_date_train: str = "1998-01-01"
    end_date_train: str = "2015-12-31"
    start_date_test: str = "2016-01-01"
    end_date_test: str = "2019-12-31"

    pca_components: int = 4          # best model in Table II
    hidden_size: int = 32
    dropout: float = 0.3
    batch_size: int = 64
    lr: float = 1e-3
    epochs: int = 500

    # For reproducibility
    seed: int = 42

def set_seed(seed: int = 42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

cfg = PaperConfig()
set_seed(cfg.seed)


In [4]:
def load_spy_data() -> pd.DataFrame:
    """
    Return a DataFrame with index = Date, columns = ['open', 'close', 'volume'].
    """
    # TODO: replace with your actual loading logic
    df = yf.download("^GSPC", start="1998-01-01", end="2021-12-31")
    df = df.rename(columns=str.lower)[['open', 'close', 'volume']]
    df.index = df.index.tz_localize(None)
    return df
    raise NotImplementedError

def load_dxy_data() -> pd.DataFrame:
    dxy_raw = yf.download("DX-Y.NYB", start="1998-01-01", end="2021-12-31", auto_adjust=False)
    df = dxy_raw.rename(columns=str.lower)[['open', 'close', 'volume']]
    df.index = df.index.tz_localize(None)
    return df.rename(columns={
        'open': 'dxy_open',
        'close': 'dxy_close',
        'volume': 'dxy_volume',
    })


def load_yield_data() -> pd.DataFrame:
    """
    Download 3M and 10Y yields from Yahoo and return a flat DataFrame:
    index = Date, columns = ['yield_3m', 'yield_10y'].
    """
    # ^IRX = 13-week T-bill (proxy for 3M), ^TNX = 10Y treasury
    tickers = {"^IRX": "yield_3m", "^TNX": "yield_10y"}

    raw = yf.download(list(tickers.keys()),
                      start="1998-01-01",
                      end="2021-12-31",
                      auto_adjust=False)

    # raw.columns: MultiIndex (field, ticker). We want Close prices.
    close = raw["Close"].copy()        # columns: ['^IRX','^TNX']

    close.columns = [tickers[c] for c in close.columns]
    close.index = close.index.tz_localize(None)

    return close      # columns: ['yield_3m', 'yield_10y']




In [10]:
def load_wsj_headlines() -> pd.DataFrame:
    df_headlines = pd.read_csv(r"C:\Users\diego\Dropbox\Proyectos\wsj_headlines.csv")
    df_headlines = df_headlines.rename(columns={"Date": "date", "Headline": "headline", "Category": "category"})
    df_headlines.index = pd.to_datetime(df_headlines["date"])
    df_headlines = df_headlines[["date","headline", "category"]]
    return df_headlines
    raise NotImplementedError


In [6]:
# Crear el cliente una sola vez (usa OPENAI_API_KEY del entorno)
client = OpenAI()

In [11]:
def embed_headline(text: str) -> np.ndarray:
    """
    Embebe un titular (o cualquier texto corto) usando text-embedding-3-small
    y devuelve un vector np.ndarray de float32.
    """
    resp = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    # resp.data[0].embedding es la lista de floats que devuelve la API
    return np.array(resp.data[0].embedding, dtype=np.float32)


In [12]:
def build_headline_embeddings(headlines_df: pd.DataFrame) -> pd.DataFrame:
    """
    headlines_df: columns ['date', 'headline']
    Returns DataFrame indexed by date with a column 'embedding' that holds np.array(1536,)
    and also expanded numeric columns if desired.
    """
    rows = []
    for _, row in headlines_df.iterrows():
        emb = embed_headline(row["headline"])
        rows.append({
            "date": row["date"],
            "embedding": emb
        })
    emb_df = pd.DataFrame(rows)
    emb_df["date"] = pd.to_datetime(emb_df["date"]).dt.normalize()
    return emb_df


In [13]:
def select_one_headline_per_day(emb_df: pd.DataFrame) -> pd.DataFrame:
    """
    emb_df: ['date', 'embedding']
    Returns DataFrame indexed by date, exactly one embedding per date.
    """
    # Randomly choose one row per date
    emb_df = emb_df.sample(frac=1.0, random_state=cfg.seed)  # shuffle
    one_per_day = emb_df.drop_duplicates(subset="date", keep="first")
    one_per_day = one_per_day.set_index("date").sort_index()
    return one_per_day


In [14]:
def fit_pca_on_train(embeddings: pd.DataFrame, cfg: PaperConfig) -> Tuple[PCA, pd.DataFrame]:
    """
    embeddings: DataFrame indexed by date, column 'embedding' (np.array)
    Returns fitted PCA and DataFrame with columns ['pca_1', ..., 'pca_k'].
    """
    # Convert list/array column into 2D matrix
    X = np.stack(embeddings["embedding"].values)  # shape (n_days, 1536)

    pca = PCA(n_components=cfg.pca_components, random_state=cfg.seed)
    X_pca = pca.fit_transform(X)

    cols = [f"headline_pca_{i+1}" for i in range(cfg.pca_components)]
    pca_df = pd.DataFrame(X_pca, index=embeddings.index, columns=cols)
    return pca, pca_df


In [15]:
def apply_pca(embeddings: pd.DataFrame, pca: PCA, cfg: PaperConfig) -> pd.DataFrame:
    X = np.stack(embeddings["embedding"].values)
    X_pca = pca.transform(X)
    cols = [f"headline_pca_{i+1}" for i in range(cfg.pca_components)]
    pca_df = pd.DataFrame(X_pca, index=embeddings.index, columns=cols)
    return pca_df

In [16]:
def build_merged_dataset(cfg: PaperConfig) -> pd.DataFrame:
    spy = load_spy_data()
    dxy = load_dxy_data()
    yld = load_yield_data()

    spy.index = pd.to_datetime(spy.index).tz_localize(None)
    dxy.index = pd.to_datetime(dxy.index).tz_localize(None)
    yld.index = pd.to_datetime(yld.index).tz_localize(None)

    spy = spy.sort_index()
    spy["log_ret_next"] = np.log(spy["close"].shift(-1) / spy["close"])

    df = spy[["open", "close", "volume", "log_ret_next"]].copy()
    df = df.join(dxy[["dxy_open"]], how="left")
    df = df.join(yld[["yield_3m", "yield_10y"]], how="left")



    # 3) Headline embeddings (assume precomputed)
    # Suppose you already have a DataFrame 'headline_embs'
    # indexed by date with column 'embedding'.
    # Here we just show the flow.
    raise NotImplementedError("You need to prepare headline_embs DataFrame")


In [41]:
def add_headline_pca_features(df: pd.DataFrame,
                              headline_embs: pd.DataFrame,
                              cfg: PaperConfig) -> Tuple[pd.DataFrame, PCA]:
    """
    df: price+macro DataFrame indexed by date
    headline_embs: DataFrame indexed by date with column 'embedding' (np.array)
    """
    # Align headline embeddings with df index
    emb_aligned = headline_embs.reindex(df.index).dropna(subset=["embedding"])

    # Fit PCA on training period only
    train_mask = (emb_aligned.index >= cfg.start_date_train) & (emb_aligned.index <= cfg.end_date_train)
    emb_train = emb_aligned[train_mask]

    pca, _ = fit_pca_on_train(emb_train, cfg)

    # Transform full (train + test) embeddings
    pca_full = apply_pca(emb_aligned, pca, cfg)   # index = dates where we have embeddings

    # Safely align PCA features to df's index and assign column by column
    pca_full_aligned = pca_full.reindex(df.index)

    for col in pca_full_aligned.columns:
        df[col] = pca_full_aligned[col]

    return df, pca


In [51]:
def flatten_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    If df.columns is a MultiIndex like ('open', '^gspc'),
    flatten it to just 'open' (first element of the tuple).
    """
    if isinstance(df.columns, pd.MultiIndex):
        df = df.copy()
        df.columns = [c[0] if isinstance(c, tuple) else c for c in df.columns]
    return df


In [52]:
def finalize_dataset(df: pd.DataFrame, cfg: PaperConfig) -> Tuple[pd.DataFrame, pd.DataFrame, StandardScaler]:
    """
    Split into train/test, handle missing data, and scale features.
    """

    # --- FLATTEN COLUMNS ---
    df = flatten_columns(df)

    # --- Debug: show columns & check target ---
    print("DEBUG finalize_dataset | columns:", list(df.columns)[:20])
    print("DEBUG finalize_dataset | 'log_ret_next' present?", "log_ret_next" in df.columns)

    # Ensure target column exists
    if "log_ret_next" not in df.columns:
        if "close" not in df.columns:
            raise ValueError(
                "finalize_dataset: df has no 'log_ret_next' and no 'close' to compute it from. "
                f"Columns are: {list(df.columns)}"
            )
        df = df.sort_index()
        df["log_ret_next"] = np.log(df["close"].shift(-1) / df["close"])
        print("DEBUG finalize_dataset | created 'log_ret_next' from 'close'.")

    # Drop rows without target
    df = df.dropna(subset=["log_ret_next"])

    # Define base feature columns
    feature_cols = [
        "open", "close", "volume",
        "dxy_open", "yield_3m", "yield_10y",
    ]

    # Add PCA columns if present
    pca_cols = [c for c in df.columns
                if isinstance(c, str) and c.startswith("headline_pca_")]
    feature_cols += pca_cols

    # Drop rows with missing features (or you could impute)
    df = df.dropna(subset=feature_cols)

    # Train/test split by date
    train_mask = (df.index >= cfg.start_date_train) & (df.index <= cfg.end_date_train)
    test_mask  = (df.index >= cfg.start_date_test)  & (df.index <= cfg.end_date_test)

    train_df = df[train_mask].copy()
    test_df  = df[test_mask].copy()

    X_train = train_df[feature_cols].values
    y_train = train_df["log_ret_next"].values

    X_test  = test_df[feature_cols].values
    y_test  = test_df["log_ret_next"].values

    # Scale features on train only
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled  = scaler.transform(X_test)

    # Store scaled arrays back into the DataFrames
    for i, col in enumerate(feature_cols):
        train_df[col] = X_train_scaled[:, i]
        test_df[col]  = X_test_scaled[:, i]

    return train_df, test_df, scaler


In [44]:
class TimeIndependentDataset(Dataset):
    def __init__(self, df: pd.DataFrame, feature_cols: list, target_col: str = "log_ret_next"):
        self.X = df[feature_cols].values.astype(np.float32)
        self.y = df[target_col].values.astype(np.float32).reshape(-1, 1)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [45]:
class FFNN(nn.Module):
    def __init__(self, input_dim: int, hidden_size: int = 32, dropout: float = 0.3):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(hidden_size, 1)

        # Xavier init
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.zeros_(self.fc1.bias)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.zeros_(self.fc2.bias)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [46]:
def smape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """
    Symmetric Mean Absolute Percentage Error in percent.
    """
    denom = (np.abs(y_true) + np.abs(y_pred))
    # avoid division by zero
    denom = np.where(denom == 0, 1e-8, denom)
    return 100.0 * np.mean(np.abs(y_true - y_pred) / denom * 2.0)


In [47]:
def train_model(model: nn.Module,
                train_loader: DataLoader,
                val_loader: DataLoader,
                cfg: PaperConfig,
                device: str = "cpu") -> Dict[str, float]:
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=cfg.lr)

    model.to(device)

    for epoch in range(cfg.epochs):
        model.train()
        train_losses = []

        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            preds = model(X_batch)
            loss = criterion(preds, y_batch)
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())

        # Optional: simple validation loss tracking
        if (epoch + 1) % 50 == 0 or epoch == 0:
            model.eval()
            val_losses = []
            with torch.no_grad():
                for X_val, y_val in val_loader:
                    X_val = X_val.to(device)
                    y_val = y_val.to(device)
                    preds = model(X_val)
                    loss = criterion(preds, y_val)
                    val_losses.append(loss.item())

            print(f"Epoch {epoch+1}/{cfg.epochs} "
                  f"Train MSE: {np.mean(train_losses):.6f} "
                  f"Val MSE: {np.mean(val_losses):.6f}")

    # Return model; metrics computed separately
    return {}


In [48]:
def evaluate_model(model: nn.Module,
                   df: pd.DataFrame,
                   feature_cols: list,
                   target_col: str = "log_ret_next",
                   device: str = "cpu") -> Dict[str, float]:
    model.eval()
    X = df[feature_cols].values.astype(np.float32)
    y_true = df[target_col].values.astype(np.float32)

    with torch.no_grad():
        X_tensor = torch.from_numpy(X).to(device)
        y_pred = model(X_tensor).cpu().numpy().flatten()

    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    s = smape(y_true, y_pred)

    return {"MSE": mse, "R2": r2, "SMAPE": s}, y_true, y_pred


In [49]:
def run_best_paper_model(cfg: PaperConfig,
                         headline_embs: pd.DataFrame | None) -> None:
    """
    headline_embs:
      - DataFrame indexed by date with column 'embedding' (np.array), or
      - None to skip news features and use only price + macro.
    """
    # 1) Base SPX + macro data
    spy = load_spy_data()
    dxy = load_dxy_data()
    yld = load_yield_data()

    # Normalize indices to tz-naive DateTimeIndex
    for frame in (spy, dxy, yld):
        if isinstance(frame.index, pd.MultiIndex):
            frame.index = frame.index.get_level_values(0)
        frame.index = pd.to_datetime(frame.index).tz_localize(None)

    spy = spy.sort_index()
    spy["log_ret_next"] = np.log(spy["close"].shift(-1) / spy["close"])

    # Base df
    df = spy[["open", "close", "volume", "log_ret_next"]].copy()

    # Add macro via aligned reindex (no joins)
    df["dxy_open"]  = dxy["dxy_open"].reindex(df.index)
    df["yield_3m"]  = yld["yield_3m"].reindex(df.index)
    df["yield_10y"] = yld["yield_10y"].reindex(df.index)

    # 2) Add PCA headline features if we have them
    if headline_embs is not None:
        df, pca = add_headline_pca_features(df, headline_embs, cfg)

    # 3) Finalize dataset (split + scaling)
    # PCA cols will be [] if we didn't add news
    pca_cols = [c for c in df.columns
                if isinstance(c, str) and c.startswith("headline_pca_")]

    feature_cols = ["open", "close", "volume",
                    "dxy_open", "yield_3m", "yield_10y"] + pca_cols

    print("DEBUG run_best_paper_model | feature_cols:", feature_cols)

    train_df, test_df, scaler = finalize_dataset(df, cfg)

    # 4) Build PyTorch datasets/loaders
    train_mask_inner = train_df.index < "2014-01-01"
    val_mask_inner   = train_df.index >= "2014-01-01"

    train_inner = train_df[train_mask_inner]
    val_inner   = train_df[val_mask_inner]

    train_dataset = TimeIndependentDataset(train_inner, feature_cols)
    val_dataset   = TimeIndependentDataset(val_inner, feature_cols)

    train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=cfg.batch_size, shuffle=False)

    # 5) Model
    input_dim = len(feature_cols)
    model = FFNN(input_dim=input_dim,
                 hidden_size=cfg.hidden_size,
                 dropout=cfg.dropout)

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # 6) Train
    train_model(model, train_loader, val_loader, cfg, device=device)

    # 7) Evaluate
    train_metrics, _, _ = evaluate_model(model, train_df, feature_cols, device=device)
    test_metrics,  _, _ = evaluate_model(model,  test_df,  feature_cols, device=device)

    print("Train metrics:", train_metrics)
    print("Test  metrics:", test_metrics)


In [25]:
df_headlines = load_wsj_headlines()
emb_df = build_headline_embeddings(df_headlines[["date", "headline"]])
one_per_day = select_one_headline_per_day(emb_df)
headline_embs = one_per_day  




In [53]:
run_best_paper_model(cfg, headline_embs=headline_embs)

C:\Users\diego\AppData\Local\Temp\ipykernel_43452\4064750746.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("^GSPC", start="1998-01-01", end="2021-12-31")
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  2 of 2 completed


DEBUG run_best_paper_model | feature_cols: ['open', 'close', 'volume', 'dxy_open', 'yield_3m', 'yield_10y']
DEBUG finalize_dataset | columns: ['open', 'close', 'volume', 'log_ret_next', 'dxy_open', 'yield_3m', 'yield_10y', 'headline_pca_1', 'headline_pca_2', 'headline_pca_3', 'headline_pca_4']
DEBUG finalize_dataset | 'log_ret_next' present? True
Epoch 1/500 Train MSE: 0.170364 Val MSE: 0.142930
Epoch 50/500 Train MSE: 0.000215 Val MSE: 0.000259
Epoch 100/500 Train MSE: 0.000176 Val MSE: 0.000173
Epoch 150/500 Train MSE: 0.000176 Val MSE: 0.000094
Epoch 200/500 Train MSE: 0.000172 Val MSE: 0.000093
Epoch 250/500 Train MSE: 0.000168 Val MSE: 0.000129
Epoch 300/500 Train MSE: 0.000169 Val MSE: 0.000263
Epoch 350/500 Train MSE: 0.000170 Val MSE: 0.000408
Epoch 400/500 Train MSE: 0.000167 Val MSE: 0.000329
Epoch 450/500 Train MSE: 0.000164 Val MSE: 0.000468
Epoch 500/500 Train MSE: 0.000166 Val MSE: 0.000400
Train metrics: {'MSE': 0.00019128543, 'R2': -0.18252894599133906, 'SMAPE': 166.494